### 필요한 라이브러리 import

In [1]:
import os
import re
import time
import requests
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from bs4 import BeautifulSoup
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

### 한글 판별 함수
기능: 문자열에 한글이 존재하면 True, 그렇지 않으면 False 반환<br>
목적: 책소개가 전부 외국어인 데이터를 제거하기 위해

In [2]:
def isKorean(string: str):
    for s in string:
        if ord('가') <= ord(s) <= ord('힣'):
            return True
    return False

### 유효 데이터 전처리
1. KDC 기호가 '문학'인 데이터 선별 (810~899)
2. 표지 url 출처가 알라딘인 데이터 선별
3. ISBN 코드가 unique한 데이터 선별
4. 책소개에 한글이 포함되어 있는 데이터 선별
5. 단권 데이터 선별

In [3]:
df = pd.read_csv('./data/Result1_Combined.csv', low_memory = False) # 병합 데이터 불러오기
print('%11s: %d행'%('전처리 전 원본', len(df)))

df['kdc_class_no'] = pd.to_numeric(df['kdc_class_no'], errors = 'coerce') # KDC 기호를 숫자형으로 변환
data = df[(df['kdc_class_no'] >= 810) & (df['kdc_class_no'] < 900)] # KDC 기호가 '문학'인 데이터 선별 (810~899)
print('%12s: %7d행'%('KDC 기호 선별 후', len(data)))

data = data[data['img_url'].notnull()] # 표지 url이 존재하지 않는 데이터 제거
data = data[data['img_url'].str.contains('image.aladin')] # 표지 url 출처가 알라딘인 데이터 선별
print('%12s: %7d행'%('표지 url 선별 후', len(data)))

data.drop_duplicates(subset = ['isbn13'], inplace = True) # ISBN 코드가 unique한 데이터 선별
print('%s: %7d행'%('ISBN 코드 선별 후', len(data)))

data = data[data['description'].notnull()] # 책소개가 존재하지 않는 데이터 제거
data = data[data['description'].map(isKorean)] # 책소개에 한글이 포함되어 있는 데이터 선별
print('%11s: %7d행'%('책소개 선별 후', len(data)))

set_idx = data[data['vol'].notnull() & data['vol'].str.contains(',|-|/')].index # 세트 데이터 인덱스
data.drop(set_idx, inplace = True) # 단권 데이터 선별 (세트 데이터 제거)
print('%12s: %7d행'%('단권 선별 후', len(data)))

   전처리 전 원본: 1257893행
 KDC 기호 선별 후:  274601행
 표지 url 선별 후:  141625행
ISBN 코드 선별 후:  118352행
   책소개 선별 후:   96731행
     단권 선별 후:   96215행


In [4]:
data.drop(['master_seq', 'pub_date', 'add_code', 'title_replace', 'author_replace', 'is_coll_aladin', 'is_coll_naver'], axis = 1, inplace = True) # 불필요한 column 제거
data.reset_index(drop = True, inplace = True) # 인덱스 초기화
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.700,2020-11-10,NaN
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.000,2020-11-06,NaN
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.800,2020-12-01,NaN
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.000,2020-09-03,NaN
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.700,2020-11-10,NaN
...,...,...,...,...,...,...,...,...,...,...,...
96210,9791160893649,NaN,영원한 소년의 정신 - 하루키 읽는 법,"양자오 (지은이), 김택규 (옮긴이)",유유,NaN,https://image.aladin.co.kr/product/27546/32/co...,세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...,830.906,2021-07-13,NaN
96211,9788960987036,NaN,오월의 편지,"정복현 (지은이), 김주리 (그림)",리틀씨앤톡,NaN,https://image.aladin.co.kr/product/27926/12/co...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...,813.800,2021-08-25,NaN
96212,9788982182921,NaN,녹색 목요일,이순임 (지은이),강,14000.0,https://image.aladin.co.kr/product/28488/96/co...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...,813.700,2021-12-17,NaN
96213,9791158543365,NaN,구름만큼 큰 대답,정미정 (엮은이),학이사어린이,11000.0,https://image.aladin.co.kr/product/28473/40/co...,"매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그...",811.800,2021-12-10,NaN


### 도서별 알라딘 ID 크롤링
ISBN 코드 검색 후 상세 페이지 링크에서 도서 ID 추출<br>
상세 페이지가 없을 경우 오류 데이터로 처리

In [5]:
id_list = [] # 알라딘 도서 ID 리스트
error_idx = [] # 오류 데이터 인덱스 리스트

for isbn in tqdm(data['isbn13']):
    search_url = 'https://www.aladin.co.kr/search/wsearchresult.aspx?SearchWord=%d'%isbn
    soup = BeautifulSoup(requests.get(search_url).text, 'html.parser')
    try:
        link = soup.select_one('a.bo3').get('href') # 알라딘 도서 상세 페이지 링크 추출
        id_list.append(re.sub(r'[^0-9]', '', link)) # 알라딘 도서 ID 추출 후 저장
    except AttributeError:
        error_idx.append(data[data['isbn13'] == isbn].index[0]) # 오류 데이터 인덱스 저장

100%|█████████████████████████████████████████████████████████████████████████| 96215/96215 [24:53:45<00:00,  1.07it/s]


In [6]:
data.drop(error_idx, inplace = True) # 오류 데이터 제거
data['item_id'] = id_list # 새로운 column 생성 후 알라딘 ID 리스트 대입
data['item_id'] = pd.to_numeric(data['item_id'], errors = 'ignore') # 알라딘 ID를 숫자형으로 변환
data.reset_index(drop = True, inplace = True) # 인덱스 초기화
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.700,2020-11-10,NaN,256477323
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.000,2020-11-06,NaN,256590129
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.800,2020-12-01,NaN,258065050
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.000,2020-09-03,NaN,240854755
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.700,2020-11-10,NaN,255000004
...,...,...,...,...,...,...,...,...,...,...,...,...
93113,9791160893649,NaN,영원한 소년의 정신 - 하루키 읽는 법,"양자오 (지은이), 김택규 (옮긴이)",유유,NaN,https://image.aladin.co.kr/product/27546/32/co...,세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...,830.906,2021-07-13,NaN,275463233
93114,9788960987036,NaN,오월의 편지,"정복현 (지은이), 김주리 (그림)",리틀씨앤톡,NaN,https://image.aladin.co.kr/product/27926/12/co...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...,813.800,2021-08-25,NaN,279261296
93115,9788982182921,NaN,녹색 목요일,이순임 (지은이),강,14000.0,https://image.aladin.co.kr/product/28488/96/co...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...,813.700,2021-12-17,NaN,284889662
93116,9791158543365,NaN,구름만큼 큰 대답,정미정 (엮은이),학이사어린이,11000.0,https://image.aladin.co.kr/product/28473/40/co...,"매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그...",811.800,2021-12-10,NaN,284734006


### 출판사 제공 책소개 크롤링

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome('./data/chromedriver', chrome_options = options)
desc_list = [] # 출판사 제공 책소개 리스트

In [8]:
for item in tqdm(data['item_id']):
    
    driver.get('https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=%d'%item)
    
    try:
        for i in range(15):
            time.sleep(0.5)
            driver.execute_script(f'window.scrollTo(0, {500 * (i + 1)})')
    except:
        desc_list.append('')
        continue
    
    desc = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1.5)
    
    try:
        desc = soup.select_one('div#div_PublisherDesc_All').text[:-3]
    except AttributeError:
        for s in reversed(soup.select('div.Ere_prod_mconts_R')):
            if '<!-- 책소개 이미지 -->' in str(s):
                desc = s.text
                break
    
    if not isKorean(desc): # 책소개에 한글이 없을 경우 빈 문자열로 대체
        desc = ''
    
    desc_list.append(desc.strip())

driver.close()

100%|███████████████████████████████████████████████████████████████████████| 93118/93118 [00:00<00:00, 2393946.56it/s]


In [9]:
data['pub_desc'] = desc_list # 새로운 column 생성 후 출판사 제공 책소개 리스트 대입
data['all_desc'] = data['description'] + data['pub_desc'] # 새로운 column 생성 후 2개의 책소개 column을 결합한 결과 대입
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.700,2020-11-10,NaN,256477323,★ 특별판 양장 에디션 한정판으로 출간 ★ \n『오전을 사는 이에게 오후도 미래다』...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.000,2020-11-06,NaN,256590129,마크 트웨인\n이브의 일기 빈티지 그림책 \n[큰글씨 한글판] \n\n* 세계 문학...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.800,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ..."
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.000,2020-09-03,NaN,240854755,NaN,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.700,2020-11-10,NaN,255000004,■ 해설 중에서\n\n숙명으로 시를 만나는 시인의 음성\n-김사달의 시집 『강촌의 ...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93113,9791160893649,NaN,영원한 소년의 정신 - 하루키 읽는 법,"양자오 (지은이), 김택규 (옮긴이)",유유,NaN,https://image.aladin.co.kr/product/27546/32/co...,세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...,830.906,2021-07-13,NaN,275463233,"일본 문학의 이방인, 하루키를 공부하는 첫걸음 \n하루키의 상상력, 삶, 세계와 작...",세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...
93114,9788960987036,NaN,오월의 편지,"정복현 (지은이), 김주리 (그림)",리틀씨앤톡,NaN,https://image.aladin.co.kr/product/27926/12/co...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...,813.800,2021-08-25,NaN,279261296,“그날 광주에 무슨 일이 있었던 거야?”\n\n모두의 동화 시리즈 20권. 어느 날...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...
93115,9788982182921,NaN,녹색 목요일,이순임 (지은이),강,14000.0,https://image.aladin.co.kr/product/28488/96/co...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...,813.700,2021-12-17,NaN,284889662,『녹색 목요일』에서 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...
93116,9791158543365,NaN,구름만큼 큰 대답,정미정 (엮은이),학이사어린이,11000.0,https://image.aladin.co.kr/product/28473/40/co...,"매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그...",811.800,2021-12-10,NaN,284734006,"어린이시, 바닷바람 맞으며 꽃을 피우다\n\n경북 울진에 사는 어린 이야기꾼들의 시...","매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그..."


### 세트 ISBN 코드 부여
목적: 동일한 세트의 도서를 구별하기 위해

In [10]:
set_path = './data/rawdata/'
files = [file for file in os.listdir(set_path) if file.startswith('NL_BO_SET')] # csv 파일명 리스트
set_isbn = pd.DataFrame()

for file in tqdm(files):
    set_isbn = pd.concat([set_isbn, pd.read_csv(set_path + file)]) # 순서대로 데이터 병합

set_isbn.drop_duplicates(subset = ['isbn13'], inplace = True) # ISBN 코드가 unique한 데이터 선별
set_isbn.set_index(keys = set_isbn['isbn13'], drop = True, inplace = True) # ISBN 코드를 인덱스로 설정
set_isbn = set_isbn[['set_isbn13']]
set_isbn

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 13.22it/s]


,set_isbn13
isbn13,
9788982412592,9788982412615
9788982412608,9788982412615
9788972783503,9788972783497
9788972783510,9788972783497
9788976829207,9788976820525
...,...
9791191766929,9791191766905
9791191766936,9791191766905
9791191766943,9791191766905


In [11]:
data_by_isbn = data.set_index(keys = data['isbn13'], drop = True)
data = data_by_isbn.join(set_isbn, how='left').reset_index(drop = True)
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.700,2020-11-10,NaN,256477323,★ 특별판 양장 에디션 한정판으로 출간 ★ \n『오전을 사는 이에게 오후도 미래다』...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,NaN
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.000,2020-11-06,NaN,256590129,마크 트웨인\n이브의 일기 빈티지 그림책 \n[큰글씨 한글판] \n\n* 세계 문학...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,NaN
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.800,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.000,2020-09-03,NaN,240854755,NaN,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,NaN
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.700,2020-11-10,NaN,255000004,■ 해설 중에서\n\n숙명으로 시를 만나는 시인의 음성\n-김사달의 시집 『강촌의 ...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93113,9791160893649,NaN,영원한 소년의 정신 - 하루키 읽는 법,"양자오 (지은이), 김택규 (옮긴이)",유유,NaN,https://image.aladin.co.kr/product/27546/32/co...,세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...,830.906,2021-07-13,NaN,275463233,"일본 문학의 이방인, 하루키를 공부하는 첫걸음 \n하루키의 상상력, 삶, 세계와 작...",세계문학공부 시리즈. 왜 사람들은 하루키에 열광할까? 하루키의 작품은 언제부터 청춘...,NaN
93114,9788960987036,NaN,오월의 편지,"정복현 (지은이), 김주리 (그림)",리틀씨앤톡,NaN,https://image.aladin.co.kr/product/27926/12/co...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...,813.800,2021-08-25,NaN,279261296,“그날 광주에 무슨 일이 있었던 거야?”\n\n모두의 동화 시리즈 20권. 어느 날...,모두의 동화 20권. 어느 날 무진이는 할머니 댁에서 부치지 못한 편지 한 통을 발...,NaN
93115,9788982182921,NaN,녹색 목요일,이순임 (지은이),강,14000.0,https://image.aladin.co.kr/product/28488/96/co...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...,813.700,2021-12-17,NaN,284889662,『녹색 목요일』에서 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게...,이순임 소설집. 작가는 조심스럽고 섬세한 촉수를 안으로 숨기고서 좀 더 과감하게 현...,NaN
93116,9791158543365,NaN,구름만큼 큰 대답,정미정 (엮은이),학이사어린이,11000.0,https://image.aladin.co.kr/product/28473/40/co...,"매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그...",811.800,2021-12-10,NaN,284734006,"어린이시, 바닷바람 맞으며 꽃을 피우다\n\n경북 울진에 사는 어린 이야기꾼들의 시...","매일 반짝임이 달라지는 울진의 바닷가, 그곳에서 정미정 동시 작가와 함께 그림을 그...",NaN


In [12]:
set_data = data[data['vol'].notnull()].sort_values(by = ['title', 'set_isbn13']) # 제목, 세트 ISBN 코드를 기준으로 정렬
set_data['set_isbn13'] = set_data['set_isbn13'].fillna(0) #결측값을 0으로 대체
set_data.reset_index(drop = True, inplace = True) # 인덱스 초기화
set_data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13
0,9791165076665,1,'너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...,저자: kiki ;옮긴이: 조민경,소미미디어,9500.0,https://image.aladin.co.kr/product/23837/59/co...,능력치 제로의 몸이면서 용사 파티에 뽑힌 플럼. 유일하게 가진 것은 ‘반전’이라는 ...,833.60,2020-05-11,NaN,238375978,저주가 축복으로 변할 때 그 절망은 반전된다―――.소녀와 소녀가 자아내는 다크라이트...,능력치 제로의 몸이면서 용사 파티에 뽑힌 플럼. 유일하게 가진 것은 ‘반전’이라는 ...,9791165076658
1,9791165079918,2,'너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...,저자: kiki ;옮긴이: 조민경,소미미디어,9500.0,https://image.aladin.co.kr/product/24938/51/co...,"모험자 길드를 좌지우지하는 데인 일당과의 싸움은 나날이 과격해졌다. 그런 가운데, ...",833.60,2020-09-01,NaN,249385186,“사람이 괴로워하는 모습을 보고 낄낄 웃다니…… 그런 쓰레기는 싹 다 죽어도 돼.”...,"모험자 길드를 좌지우지하는 데인 일당과의 싸움은 나날이 과격해졌다. 그런 가운데, ...",9791165076658
2,9791166114052,3,'너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...,저자: kiki ;옮긴이: 조민경,소미미디어,11000.0,https://image.aladin.co.kr/product/25972/15/co...,"플럼의 전 파티 멤버이자, 지금은 검술의 스승인 가디오의 앞에 사별했을 아내 티아가...",833.60,2021-01-15,NaN,259721568,"“각오는 됐어? 안 됐어도 할 거지만.” 플럼의 전 파티 멤버이자, 지금은 검술의 ...","플럼의 전 파티 멤버이자, 지금은 검술의 스승인 가디오의 앞에 사별했을 아내 티아가...",9791165076658
3,9791196146429,2,"'촌티'나게 살았소 :소박한 삶에서 나오는 담백, 구수한 시·수필집","지은이: 김정식,전영실",동푼,15000.0,https://image.aladin.co.kr/product/19322/75/co...,2018년 처음으로 농촌재능나눔 사업에 선정되어 청양 신대2리에서 시골 할머니 할아...,810.82,2019-04-23,NaN,193227588,"‘촌티나게 살았소!’\n\n왜, 갑자기 「촌티학교」라고 하고 싶었을까? \n\n지난...",2018년 처음으로 농촌재능나눔 사업에 선정되어 청양 신대2리에서 시골 할머니 할아...,0
4,9788914017345,1,(1001가지 소원을 들어주는) 미니마와 막시마,이자벨 아베디 글 ;참다래 그림,삼성당,11000.0,http://image.aladin.co.kr/product/726/65/cover...,판타지 요소를 가미한 성장소설. 레기날드의 세계는 탁자 위의 건축물들과 각 탁자를 ...,853.00,2010-06-25,9788914017345(1),7266599,"판타지의 한계를 넘어선 기상천외한 미니어처 세계 \n\n피라미드, 자유의 여신상, ...",판타지 요소를 가미한 성장소설. 레기날드의 세계는 탁자 위의 건축물들과 각 탁자를 ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51036,9788953238091,17,힙합=Hip hop,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/39/41/cover...,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",810.00,2002-12-31,9788953238091,394176,NaN,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",0
51037,9788953244214,20,힙합=Hip hop,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/42/60/cover...,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,810.00,2003-07-30,9788953244214,426012,NaN,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,0
51038,9788952967725,37,龍,Motoka Murakami 저;余影兒 譯,학산문화사,3800.0,https://image.aladin.co.kr/product/55/59/cover...,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,830.00,2005-04-18,8952967724(37),555997,NaN,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,0
51039,9788990028129,상,李歐=Liou,다카무라 카오루 지음;김소연 譯,손안의책,6800.0,http://image.aladin.co.kr/product/40/2/cover/8...,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",833.60,2003-02-15,8990028124(상),400289,NaN,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",0


In [13]:
prev, code = set_data.iloc[len(set_data) - 1], -1
set_data['set_isbn13'][len(set_data) - 1] = code

for i in tqdm(range(len(set_data) - 2, -1, -1)):
    target = set_data.iloc[i]
    if target['set_isbn13']:
        continue
    if (prev['title'], prev['author']) == (target['title'], target['author']):
        set_data['set_isbn13'][i] = set_data['set_isbn13'][i + 1]
    else:
        code -= 1
        set_data['set_isbn13'][i] = code
        prev = target

100%|██████████████████████████████████████████████████████████████████████████| 51040/51040 [00:13<00:00, 3891.08it/s]


In [14]:
set_data['title'] = set_data[['title', 'vol']].agg(' '.join, axis = 1) # 제목과 권수를 결합하여 제목 column에 대입
set_data.title

0        '너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...
1        '너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...
2        '너 따위가 마왕을 이길 수 있다고 생각하지마'라며 용사 파티에서 추방되었으니 왕도...
3                  '촌티'나게 살았소 :소박한 삶에서 나오는 담백, 구수한 시·수필집 2
4                             (1001가지 소원을 들어주는) 미니마와 막시마 1
                               ...                        
51036                                        힙합=Hip hop 17
51037                                        힙합=Hip hop 20
51038                                                 龍 37
51039                                            李歐=Liou 상
51040                                            李歐=Liou 하
Name: title, Length: 51041, dtype: object

In [15]:
data = pd.concat([data[data['vol'].isnull()], set_data]) # 기존 데이터와 시리즈 데이터를 결합
data.reset_index(drop = True, inplace = True) # 인덱스 초기화
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.7,2020-11-10,NaN,256477323,★ 특별판 양장 에디션 한정판으로 출간 ★ \n『오전을 사는 이에게 오후도 미래다』...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,NaN
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.0,2020-11-06,NaN,256590129,마크 트웨인\n이브의 일기 빈티지 그림책 \n[큰글씨 한글판] \n\n* 세계 문학...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,NaN
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.0,2020-09-03,NaN,240854755,NaN,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,NaN
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.7,2020-11-10,NaN,255000004,■ 해설 중에서\n\n숙명으로 시를 만나는 시인의 음성\n-김사달의 시집 『강촌의 ...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93113,9788953238091,17,힙합=Hip hop 17,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/39/41/cover...,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",810.0,2002-12-31,9788953238091,394176,NaN,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",-3
93114,9788953244214,20,힙합=Hip hop 20,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/42/60/cover...,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,810.0,2003-07-30,9788953244214,426012,NaN,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,-3
93115,9788952967725,37,龍 37,Motoka Murakami 저;余影兒 譯,학산문화사,3800.0,https://image.aladin.co.kr/product/55/59/cover...,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,830.0,2005-04-18,8952967724(37),555997,NaN,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,-2
93116,9788990028129,상,李歐=Liou 상,다카무라 카오루 지음;김소연 譯,손안의책,6800.0,http://image.aladin.co.kr/product/40/2/cover/8...,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",833.6,2003-02-15,8990028124(상),400289,NaN,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",-1


### 책소개 전처리
책소개 텍스트에서 유효한 명사 추출

In [16]:
data = pd.read_csv('./data/Result2_Preprocessed.csv', low_memory = False)
okt = Okt()
data['desc_noun'] = [' '.join(nouns) for nouns in tqdm(map(okt.nouns, data['all_desc']))]
data.desc_noun

93118it [53:41, 28.90it/s] 


0        오전 이 오후 미래 간 이후 독자 입 소문 타고 사랑 시민 직접 투표 북원 부산 올...
1        마크 트웨인 이브 일기 빈티 그림책 글씨 한글 판 세계 문학 명작 소설 어른 위 동...
2        이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
3        한밤중 세상 모든 존재 전하 우리 앞 보고 달리 코로나 사태 나니 서서 것 나 혼자...
4        청어 인선 권 달 시집 달 시 특징 언어 사용 매우 시어 결합 재치 언어 사용 긴축...
                               ...                        
93113    국내 최초 본격 힙합댄스 만화 연예인 가수 주인공 만화 이 만화 조금 직업 춤 의 ...
93114    어머니 사랑 제대로 못 아버지 꼴통 태 날 자신 젊음 수 그것 바로 춤 성태 그 친...
93115    의 만화 스 파스 스 파스 제 권 아츠시 악령 퇴 치사 가문 후손 악령 홀려 세 소...
93116    나오키상 추리 작가 협회 등 수상한 미스테리 여왕 다카 무 카오루 섬세 청춘 소설 ...
93117    나오키상 추리 작가 협회 등 수상한 미스테리 여왕 다카 무 카오루 섬세 청춘 소설 ...
Name: desc_noun, Length: 93118, dtype: object

In [17]:
data.info() # 데이터 정보 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93118 entries, 0 to 93117
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   isbn13        93118 non-null  int64  
 1   vol           51041 non-null  object 
 2   title         93118 non-null  object 
 3   author        93118 non-null  object 
 4   publisher     93118 non-null  object 
 5   price         90995 non-null  float64
 6   img_url       93118 non-null  object 
 7   description   93118 non-null  object 
 8   kdc_class_no  93118 non-null  float64
 9   pub_date_2    93118 non-null  object 
 10  isbn_origin   55434 non-null  object 
 11  item_id       93118 non-null  int64  
 12  pub_desc      51463 non-null  object 
 13  all_desc      93118 non-null  object 
 14  set_isbn13    51045 non-null  float64
 15  desc_noun     93118 non-null  object 
dtypes: float64(3), int64(2), object(11)
memory usage: 11.4+ MB


In [18]:
data_path = './data/Result2_Preprocessed.csv'
data.to_csv(data_path, encoding = 'utf-8-sig', index = None) # csv로 저장

In [19]:
pd.read_csv(data_path) # 데이터 불러와서 확인

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13,desc_noun
0,9788965456780,NaN,오전을 사는 이에게 오후도 미래다 (특별판 양장 에디션),이국환 (지은이),산지니,16000.0,https://image.aladin.co.kr/product/25647/73/co...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,814.7,2020-11-10,NaN,256477323,★ 특별판 양장 에디션 한정판으로 출간 ★ \n『오전을 사는 이에게 오후도 미래다』...,&lt;오전을 사는 이에게 오후도 미래다&gt;는 출간 이후 독자들의 입소문을 타고...,NaN,오전 이 오후 미래 간 이후 독자 입 소문 타고 사랑 시민 직접 투표 북원 부산 올...
1,9791137222472,NaN,"마크 트웨인, 이브의 일기 빈티지 그림책 (큰글씨 한글판)",마크 트웨인 (지은이),부크크(bookk),9200.0,https://image.aladin.co.kr/product/25659/1/cov...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,843.0,2020-11-06,NaN,256590129,마크 트웨인\n이브의 일기 빈티지 그림책 \n[큰글씨 한글판] \n\n* 세계 문학...,마크 트웨인이브의 일기 빈티지 그림책 [큰글씨 한글판] * 세계 문학 명작 소설* ...,NaN,마크 트웨인 이브 일기 빈티 그림책 글씨 한글 판 세계 문학 명작 소설 어른 위 동...
2,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN,이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
3,9781843654650,NaN,While You're Sleeping (Hardcover),"Mick Jackson (지은이), John Broadley (그림)",Pavilion Books,24190.0,https://image.aladin.co.kr/product/24085/47/co...,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,843.0,2020-09-03,NaN,240854755,NaN,한밤중 세상을 움직이는 모든 존재들에게 전하는 헌사!&#x0D;&#x0D;우리는 앞...,NaN,한밤중 세상 모든 존재 전하 우리 앞 보고 달리 코로나 사태 나니 서서 것 나 혼자...
4,9791158609047,NaN,강촌의 사계,김사달 (지은이),청어,10000.0,https://image.aladin.co.kr/product/25500/0/cov...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",811.7,2020-11-10,NaN,255000004,■ 해설 중에서\n\n숙명으로 시를 만나는 시인의 음성\n-김사달의 시집 『강촌의 ...,"청어시인선 263권. 김사달 시집. 김사달 시의 특징은 언어 사용이 매우 유연하며,...",NaN,청어 인선 권 달 시집 달 시 특징 언어 사용 매우 시어 결합 재치 언어 사용 긴축...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93113,9788953238091,17,힙합=Hip hop 17,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/39/41/cover...,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",810.0,2002-12-31,9788953238091,394176,NaN,"국내 최초의 본격 힙합댄스 만화. 연예인 가수를 주인공으로 한 만화는 많지만, 이 ...",-3.0,국내 최초 본격 힙합댄스 만화 연예인 가수 주인공 만화 이 만화 조금 직업 춤 의 ...
93114,9788953244214,20,힙합=Hip hop 20,김수용 글·그림,서울문화사,3000.0,https://image.aladin.co.kr/product/42/60/cover...,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,810.0,2003-07-30,9788953244214,426012,NaN,어머니의 사랑을 제대로 받지 못하고 아버지와 함께 자란 꼴통 태하. 어느날 자신의 ...,-3.0,어머니 사랑 제대로 못 아버지 꼴통 태 날 자신 젊음 수 그것 바로 춤 성태 그 친...
93115,9788952967725,37,龍 37,Motoka Murakami 저;余影兒 譯,학산문화사,3800.0,https://image.aladin.co.kr/product/55/59/cover...,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,830.0,2005-04-18,8952967724(37),555997,NaN,TAISHI MIYAKE의 만화 『스파스 스파스』 제3권. 아츠시는 악령퇴치사 가문...,-2.0,의 만화 스 파스 스 파스 제 권 아츠시 악령 퇴 치사 가문 후손 악령 홀려 세 소...
93116,9788990028129,상,李歐=Liou 상,다카무라 카오루 지음;김소연 譯,손안의책,6800.0,http://image.aladin.co.kr/product/40/2/cover/8...,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",833.6,2003-02-15,8990028124(상),400289,NaN,"나오키상, 일본추리작가협회상 등을 수상한 미스테리의 여왕 다카무라 카오루의 섬세하고...",-1.0,나오키상 추리 작가 협회 등 수상한 미스테리 여왕 다카 무 카오루 섬세 청춘 소설 ...


### 한국 소설 분류
KDC 기호가 한국 소설인 데이터 추출

In [20]:
data = data[(data['kdc_class_no'] >= 813) & (data['kdc_class_no'] < 814)].reset_index(drop = True)
data

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13,desc_noun
0,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN,이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
1,9791186259948,NaN,어쩜 좋아!,박은율 (지은이),시시한출판사,20000.0,https://image.aladin.co.kr/product/25964/92/co...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,813.8,2020-12-24,NaN,259649242,그림책 <어쩜 좋아!>는 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림책-...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,NaN,어린이 그림책 권 출판사 주최 그림책 공모전 어린이 그림책 부문 선정 작품 비밀 약...
2,9788936433444,NaN,괭이부리말 아이들,김중미 지음;송진헌 그림,창작과 비평사,12000.0,http://image.aladin.co.kr/product/30/41/cover/...,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,813.6,2001-11-07,893643344X:,304191,NaN,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,NaN,두 권 의 양장본 제 어린이 책 원고 공모 창작 부문 대상 수상작 이 작품 방송 오...
3,9788951508752,NaN,꿈꾸는 동글이,동화창의학습연구회 편;김숙경 [공]그림,동화사,7000.0,http://image.aladin.co.kr/product/29/37/cover/...,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",813.6,2001-08-24,8951508757,293751,NaN,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",NaN,새 교과서 말 듣기 쓰기 창의 학습 동화 국어 수학 과학 사회 도덕 등 모든 과목 ...
4,9788956511368,NaN,서울 여자와 대구 남자가 만날 때,샤베트 지음,영상노트,9000.0,http://image.aladin.co.kr/product/42/5/cover/8...,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,813.6,2003-06-27,8956511365,420501,정민 로맨스 소설.,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,NaN,참여정부 행정 수도 한강 남쪽 지역 것 공약 때 서울 태생 서울 작가 서울 애정 가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32318,9791189824051,5,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 5,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/18011/35/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-01-02,NaN,180113502,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32319,9791164242979,9,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 9,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/19362/34/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,813.7,2019-05-29,NaN,193623473,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32320,9791164245963,11,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 11,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20122/76/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-07-31,NaN,201227638,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32321,9791136500687,12,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 12,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20646/2/cov...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-09-03,NaN,206460246,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...


In [21]:
data_path = './data/Result3_Preprocessed.csv'
data.to_csv(data_path, encoding = 'utf-8-sig', index = None) # csv로 저장

In [22]:
pd.read_csv(data_path) # 데이터 불러와서 확인

,isbn13,vol,title,author,publisher,price,img_url,description,kdc_class_no,pub_date_2,isbn_origin,item_id,pub_desc,all_desc,set_isbn13,desc_noun
0,9791195991341,NaN,The Secret Under the Earth - &lt;이 땅의 비밀&gt; 영문판,"서정아 (지은이), 오상우 (그림), 이동권 (해설)",이야기뜰,15000.0,https://image.aladin.co.kr/product/25806/50/co...,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",813.8,2020-12-01,NaN,258065050,NaN,"&lt;이 땅의 비밀&gt; 영문판. 산 중턱, 층층이 쌓인 바위 사이에 돌이 된 ...",NaN,이 땅 비밀 영문 판 산 중턱 층층 바위 사이 돌이 물고기 이 물고기 아이 물고기 ...
1,9791186259948,NaN,어쩜 좋아!,박은율 (지은이),시시한출판사,20000.0,https://image.aladin.co.kr/product/25964/92/co...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,813.8,2020-12-24,NaN,259649242,그림책 <어쩜 좋아!>는 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림책-...,어린이가 펴낸 그림책 4권. 시시한출판사가 주최한 그림책 공모전(어린이가 펴낸 그림...,NaN,어린이 그림책 권 출판사 주최 그림책 공모전 어린이 그림책 부문 선정 작품 비밀 약...
2,9788936433444,NaN,괭이부리말 아이들,김중미 지음;송진헌 그림,창작과 비평사,12000.0,http://image.aladin.co.kr/product/30/41/cover/...,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,813.6,2001-11-07,893643344X:,304191,NaN,2000년 7월 두 권으로 선보였던 의 양장본. 제4회 '좋은 어린이책' 원고공모 ...,NaN,두 권 의 양장본 제 어린이 책 원고 공모 창작 부문 대상 수상작 이 작품 방송 오...
3,9788951508752,NaN,꿈꾸는 동글이,동화창의학습연구회 편;김숙경 [공]그림,동화사,7000.0,http://image.aladin.co.kr/product/29/37/cover/...,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",813.6,2001-08-24,8951508757,293751,NaN,"새 교과서의 말하기, 듣기, 읽기, 쓰기에 맞춘 창의학습동화. 국어, 수학, 과학,...",NaN,새 교과서 말 듣기 쓰기 창의 학습 동화 국어 수학 과학 사회 도덕 등 모든 과목 ...
4,9788956511368,NaN,서울 여자와 대구 남자가 만날 때,샤베트 지음,영상노트,9000.0,http://image.aladin.co.kr/product/42/5/cover/8...,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,813.6,2003-06-27,8956511365,420501,정민 로맨스 소설.,새로 들어선 참여정부가 행정수도를 한강 남쪽지역으로 옮길 것이라는 공약을 내놓은 이...,NaN,참여정부 행정 수도 한강 남쪽 지역 것 공약 때 서울 태생 서울 작가 서울 애정 가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32318,9791189824051,5,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 5,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/18011/35/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-01-02,NaN,180113502,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32319,9791164242979,9,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 9,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/19362/34/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,813.7,2019-05-29,NaN,193623473,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고 그냥 호구인 ...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32320,9791164245963,11,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 11,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20122/76/co...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-07-31,NaN,201227638,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
32321,9791136500687,12,힐통령 태양의 사제 :제리엠 게임판타지 장편소설 12,지은이: 제리엠,예원북스,8000.0,https://image.aladin.co.kr/product/20646/2/cov...,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,813.7,2019-09-03,NaN,206460246,NaN,&quot;착하긴 뭐가 착해? 저런 퀘스트를 하는 건 착해서가 아니고그냥 호구인 거...,9.791189e+12,뭐 저런 퀘스트 건 그냥 호구 호구 등 뒤 소리 카이 그 내 호구 설마 곤경 처 선...
